# Project 1


## Description

## Data import

In [92]:
import dlc_practical_prologue as prologue

# We import 1000 pairs of digits for the training and the testing inputs, targets and classes
number_of_pairs = 1000 
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets( 
    number_of_pairs)

# Quick verification of the sizes:
print('Size of train_input:', train_input.size())
print('Size of train_target:', test_target.size())
print('Size of train_classes:', test_classes.size())
print('Size of test_input:', test_input.size())
print('Size of test_target:', test_target.size())
print('Size of test_classes:', test_classes.size())



Size of train_input: torch.Size([1000, 2, 14, 14])
Size of train_target: torch.Size([1000])
Size of train_classes: torch.Size([1000, 2])
Size of test_input: torch.Size([1000, 2, 14, 14])
Size of test_target: torch.Size([1000])
Size of test_classes: torch.Size([1000, 2])


## CNN

In [93]:
import torch
from torch import nn
from torch.nn import functional as F

In [94]:
class ConvNet(nn.Module):
    def __init__(self, initial_layers=2, final_layers=2):
        super().__init__()
        self.conv1 = nn.Conv2d(initial_layers, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, final_layers)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

## CNN with Weight Sharing

#### We define the same class wether we use auxiliary loss or not

In [95]:
class WeightSharing(nn.Module):
    def __init__(self, auxiliary_loss = False):
        super().__init__()
        self.sharedConvNet = ConvNet(initial_layers=1, final_layers=10)
        self.auxiliary_loss = auxiliary_loss

        self.fc1 = nn.Linear(20,100)
        self.fc2 = nn.Linear(100,20)
        self.fc3 = nn.Linear(20,2)
    
    def forward(self, x):
        image1 = x.narrow(1,0,1)
        image2 = x.narrow(1,1,1)

        output1 = self.sharedConvNet(image1)
        output2 = self.sharedConvNet(image2)
        outputCat = torch.cat((output1, output2), 1)

        outputCat = F.relu(self.fc1(outputCat))
        outputCat = F.relu(self.fc2(outputCat))
        outputCat = self.fc3(outputCat)

        
        if self.auxiliary_loss:
            return outputCat, output1, output2
        else:
            return outputCat

## Model training and number of errors

In [96]:
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 50, lr = 5e-1, printAccLoss = False, auxiliary_loss = False, alpha = 0.3, beta = 0.5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)

    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            if auxiliary_loss:
                trainTargetImage1 = (train_classes.narrow(1,0,1)).squeeze()
                trainTargetImage2 = (train_classes.narrow(1,1,1)).squeeze()

                outputCat, output1, output2 = model(train_input.narrow(0, b, mini_batch_size))
                lossCat = criterion(outputCat, train_target.narrow(0, b, mini_batch_size))
                loss1 = criterion(output1, trainTargetImage1.narrow(0, b, mini_batch_size))
                loss2 = criterion(output2, trainTargetImage2.narrow(0, b, mini_batch_size))

                loss = alpha*loss1 + alpha*loss2 + beta*lossCat

            else:
                output = model(train_input.narrow(0, b, mini_batch_size))
                loss = criterion(output, train_target.narrow(0, b, mini_batch_size))

            acc_loss = acc_loss + loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        if printAccLoss:
            print(e, acc_loss)

def compute_nb_errors(model, input, target, mini_batch_size, auxiliary_loss = False):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        if auxiliary_loss:
            outputCat, _, _ = model(input.narrow(0, b, mini_batch_size))
            _, predicted_classes = outputCat.max(1)
            for k in range(mini_batch_size):
                if target[b + k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1
        else:
            output = model(input.narrow(0, b, mini_batch_size))
            _, predicted_classes = output.max(1)
            for k in range(mini_batch_size):
                if target[b + k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1


    return nb_errors

## Models testing

In [97]:
import time
mini_batch_size = 50
number_of_runs = 10

def test_model(model, number_of_runs=10, print_times=False, print_accuracy=False, auxiliary_loss=False):
    train_error_ratios = []
    test_error_ratios = []
    for run in range(number_of_runs):
        train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(number_of_pairs)

        if print_times: startTime = time.time()
        model.train(True)
        train_model(model, train_input, train_target, mini_batch_size, auxiliary_loss=auxiliary_loss)
        model.train(False)

        if print_times: endTime = time.time()

        nb_train_errors = compute_nb_errors(model, train_input, train_target, mini_batch_size,auxiliary_loss=auxiliary_loss)
        nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size, auxiliary_loss=auxiliary_loss)
        train_error_ratios.append(nb_train_errors/train_input.size(0))
        test_error_ratios.append(nb_test_errors/test_input.size(0))

        if print_accuracy:
            print('--- Run', run, '---')
            print('train accuracy:', nb_train_errors/train_input.size(0))
            print('test accuracy:', nb_test_errors/test_input.size(0))
        if print_times: print('Time for training:', str(endTime-startTime))
    return train_error_ratios, test_error_ratios



### CNN

In [98]:
train_error_ratios, test_error_ratios = test_model(ConvNet(), print_accuracy=True)

--- Run 0 ---
train accuracy: 0.451
test accuracy: 0.432


KeyboardInterrupt: 

### CNN with Weight Sharing

In [ ]:
train_error_ratios, test_error_ratios = test_model(WeightSharing(), print_accuracy=True)

test error Net 43.50% 435/1000
Time for model: 0.002001047134399414, time for training: 8.408761262893677
test error Net 54.30% 543/1000
Time for model: 0.003000020980834961, time for training: 8.754936933517456
test error Net 57.10% 571/1000
Time for model: 0.0019998550415039062, time for training: 8.688370943069458
test error Net 54.40% 544/1000
Time for model: 0.002000570297241211, time for training: 8.580798149108887
test error Net 54.10% 541/1000
Time for model: 0.0020012855529785156, time for training: 8.453062772750854
test error Net 44.50% 445/1000
Time for model: 0.0029997825622558594, time for training: 8.862008571624756
test error Net 45.70% 457/1000
Time for model: 0.0029993057250976562, time for training: 8.709026098251343
test error Net 46.50% 465/1000
Time for model: 0.002984285354614258, time for training: 8.891265392303467
test error Net 53.60% 536/1000
Time for model: 0.003000020980834961, time for training: 8.855331420898438
test error Net 54.60% 546/1000
Time for mo

### CNN with Weight Sharing and Auxiliary Loss

In [ ]:
train_error_ratios, test_error_ratios = test_model(WeightSharing(auxiliary_loss=True), auxiliary_loss=True, print_accuracy=True)

test error Net 55.60% 556/1000
Time for model: 0.0020003318786621094, time for training: 8.730224847793579
test error Net 54.20% 542/1000
Time for model: 0.0020003318786621094, time for training: 8.711737632751465
test error Net 52.80% 528/1000
Time for model: 0.0020003318786621094, time for training: 9.527181148529053
test error Net 56.30% 563/1000
Time for model: 0.0020008087158203125, time for training: 9.174331903457642
test error Net 55.40% 554/1000
Time for model: 0.0020003318786621094, time for training: 9.293709754943848
test error Net 55.70% 557/1000
Time for model: 0.0020003318786621094, time for training: 8.628757238388062
test error Net 56.50% 565/1000
Time for model: 0.002998828887939453, time for training: 8.669739961624146
test error Net 54.10% 541/1000
Time for model: 0.0019843578338623047, time for training: 8.568743705749512
test error Net 55.70% 557/1000
Time for model: 0.002000093460083008, time for training: 8.942838430404663
test error Net 57.00% 570/1000
Time for